<h1 align="center">
  <a href="https://portkey.ai">
    <img width="300" src="https://analyticsindiamag.com/wp-content/uploads/2023/08/Logo-on-white-background.png" alt="portkey">
  </a>
</h1>

[Portkey](https://app.portkey.ai/) is the Control Panel for AI apps. With it's popular AI Gateway and Observability Suite, hundreds of teams ship reliable, cost-efficient, and fast apps.

With Portkey, you can

- Connect to 200+ models through a unified API,
- View 40+ metrics & logs for all requests
- Enable semantic cache to reduce latency & costs
- Implement automatic retries & fallbacks for failed requests

This notebook demonstrates how to use Portkey for function calling with various LLM providers including OpenAI, Anthropic, and Google's Gemini.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1TeqYiXhlfZvFhoOHWEiyBycfWu0arWwp?usp=sharing)

In [ ]:
!pip install -qU openai portkey-ai

In [ ]:
tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]

In [ ]:
restaurant_recommendations = [{
        "type": "function",
        "function": {
            "name": "get_restaurant_recommendations",
            "description": "Get restaurant recommendations for a given location and cuisine type",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. New York, NY"
                    },
                    "cuisine": {
                        "type": "string",
                        "description": "The type of cuisine, e.g. Italian, Chinese, Mexican"
                    },
                    "price_range": {
                        "type": "string",
                        "enum": ["$", "$$", "$$$", "$$$$"],
                        "description": "The price range of the restaurants"
                    }
                },
                "required": ["location", "cuisine"]
            }
        }
    },
    ]

In [ ]:
latest_ai_news =  [{
    "type": "function",
    "function": {
        "name": "get_latest_ai_news",
        "description": "Retrieve the latest news articles about artificial intelligence",
        "parameters": {
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "Specific AI topic (optional, e.g., 'machine learning', 'neural networks', 'robotics')"
                },
                "timeframe": {
                    "type": "string",
                    "enum": ["today", "this_week", "this_month"],
                    "description": "Time period for the news articles"
                },
                "source_type": {
                    "type": "string",
                    "enum": ["all", "academic", "industry", "mainstream"],
                    "description": "Type of news sources to include"
                },
                "max_results": {
                    "type": "integer",
                    "description": "Maximum number of news articles to return (1-50)",
                    "minimum": 1,
                    "maximum": 50
                },
                "language": {
                    "type": "string",
                    "description": "Preferred language for the articles (e.g., 'en' for English, 'es' for Spanish)"
                }
            },
            "required": ["timeframe"]
        }
    }
}]

In [ ]:
# Initialize Portkey client
from portkey_ai import Portkey
from portkey_ai import PORTKEY_GATEWAY_URL, createHeaders
from google.colab import userdata
from tabulate import tabulate

providers = [
        ("openai", "gpt-4o"),
         ("openai", "gpt-4o-mini"),
        ("anthropic", "claude-3-5-sonnet-20240620"),
        ("anthropic", "claude-3-opus-20240229"),
        ("anthropic", "claude-3-haiku-20240307"),
        ("google", "gemini-1.5-flash-latest"),
        ("google", "gemini-1.5-pro"),
        #("fireworks-ai", "accounts/yi-01-ai/models/yi-large"),
        ("fireworks-ai", "accounts/fireworks/models/firefunction-v2"),
        # ("fireworks-ai", "accounts/fireworks/models/codegemma-2b"),
        #  ("fireworks-ai", "accounts/fireworks/models/llama-v2-7b"),
        #("groq", "llama3-groq-70b-8192-tool-use-preview"),

   #  ("together-ai", "meta-llama/Llama-2-70b-hf"),  #-  Error code: 400 - {'error': {'message': 'together-ai error: google/gemma-2-27b-it is not supported for JSON mode/function calling


    ]


portkey = Portkey(api_key=userdata.get('PORTKEY_API_KEY'))


def portkey_function_call(messages):
    results = []

    for provider, model in providers:
        config = {
            "provider": provider.lower(),
            "api_key": userdata.get(f'{provider.upper()}_API_KEY')
        }


        # Make the API call
        response = portkey.with_options(config=config).chat.completions.create(
            model=model,
            messages=messages,
            tools=latest_ai_news,
            tool_choice="auto",
            max_tokens=512,
        )

        response_message = response.choices[0].message
        tool_call_response = str(response_message.tool_calls) if response_message.tool_calls else "No tool calls"

        results.append([model, provider, tool_call_response])

    # Print results using tabulate
    print(tabulate(results, headers=["Model", "Provider", "Tool Call Response"], tablefmt="grid"))

#Testing of Get Latest AI News Tool

In [ ]:
# Test message
messages = [{"role": "user", "content": "give me latest ai news of this week"}]

# Run function calling for all providers
portkey_function_call(messages)

+-------------------------------------------+--------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Model                                     | Provider     | Tool Call Response                                                                                                                                                                                                                       |
+===========================================+==============+==========================================================================================================================================================================================================================================+
| gpt-4o                                    | openai       | [ChatCompletionMessageToolCall(id='call_9zk75e22eO5

#Testing of restaurant_recommendations Tool

In [ ]:
# Test message
messages = [{"role": "user", "content": "Suggest latest and best restaurant of Aurangabad?"}]

# Run function calling for all providers
portkey_function_call(messages)

+-------------------------------------------+--------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Model                                     | Provider     | Tool Call Response                                                                                                                                                                                                                                              |
+===========================================+==============+=================================================================================================================================================================================================================================================================+
| gpt-4o                                   

#Testing of Current Weather Featching Tool

In [ ]:
# Test message
messages = [{"role": "user", "content": "How's the weather like in San Francisco?"}]

# Run function calling for all providers
portkey_function_call(messages)

+-------------------------------------------+--------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Model                                     | Provider     | Tool Call Response                                                                                                                                                                                                                                                                            |
+===========================================+==============+==================================================================================================================================================================================================================================================

In [ ]:
# Test message
messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]

# Run function calling for all providers
portkey_function_call(messages)

+-------------------------------------------+--------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Model                                     | Provider     | Tool Call Response                                                                                                                                                                                   

In [ ]:
# Test message
messages = [{"role": "user", "content": "What's the weather like in Mumbai ,Delhi and Banglore?"}]

# Run function calling for all providers
portkey_function_call(messages)

+-------------------------------------------+--------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Model                                     | Provider     | Tool Call Response                                                                                                                  